In [4]:
import pandas as pd
import numpy as np
import os
import pickle

In [11]:
raw_data_path = "../repos/Temporal_Relational_Stock_Ranking/data/google_finance/"
data_universe_path = "../repos/Temporal_Relational_Stock_Ranking/data/2013-01-01"
output_data_path = "../data/"

In [15]:
exchanges = ["NASDAQ"]
filenames = []
for exchange in exchanges:
    filenames += sorted(map(lambda x: x.replace("_1", "_30Y"), filter(lambda s: s.startswith(exchange+"_"), os.listdir(data_universe_path))))
filenames

['NASDAQ_AABA_30Y.csv',
 'NASDAQ_AAON_30Y.csv',
 'NASDAQ_AAPL_30Y.csv',
 'NASDAQ_AAWW_30Y.csv',
 'NASDAQ_AAXJ_30Y.csv',
 'NASDAQ_AAXN_30Y.csv',
 'NASDAQ_ABAX_30Y.csv',
 'NASDAQ_ABCB_30Y.csv',
 'NASDAQ_ABCO_30Y.csv',
 'NASDAQ_ABMD_30Y.csv',
 'NASDAQ_ACET_30Y.csv',
 'NASDAQ_ACGL_30Y.csv',
 'NASDAQ_ACHC_30Y.csv',
 'NASDAQ_ACIW_30Y.csv',
 'NASDAQ_ACOR_30Y.csv',
 'NASDAQ_ACTA_30Y.csv',
 'NASDAQ_ACWI_30Y.csv',
 'NASDAQ_ACWX_30Y.csv',
 'NASDAQ_ACXM_30Y.csv',
 'NASDAQ_ADBE_30Y.csv',
 'NASDAQ_ADI_30Y.csv',
 'NASDAQ_ADP_30Y.csv',
 'NASDAQ_ADRD_30Y.csv',
 'NASDAQ_ADRE_30Y.csv',
 'NASDAQ_ADSK_30Y.csv',
 'NASDAQ_ADTN_30Y.csv',
 'NASDAQ_ADUS_30Y.csv',
 'NASDAQ_AEGN_30Y.csv',
 'NASDAQ_AEIS_30Y.csv',
 'NASDAQ_AFAM_30Y.csv',
 'NASDAQ_AFSI_30Y.csv',
 'NASDAQ_AGII_30Y.csv',
 'NASDAQ_AGNC_30Y.csv',
 'NASDAQ_AGYS_30Y.csv',
 'NASDAQ_AHGP_30Y.csv',
 'NASDAQ_AIA_30Y.csv',
 'NASDAQ_AIMC_30Y.csv',
 'NASDAQ_AKAM_30Y.csv',
 'NASDAQ_AKRX_30Y.csv',
 'NASDAQ_ALCO_30Y.csv',
 'NASDAQ_ALGN_30Y.csv',
 'NASDAQ_ALGT_30Y.c

In [86]:
sim_period = ["2010-01-01", "2017-12-08"]
data_names = ["Open", "High", "Low", "Close", "Volume"]
data = dict()

# Generate delay-0 data
i = 0
for filename in filenames:
    i += 1
    if i % 100 == 0:
        print(f"Processing {i} / {len(filenames)} files...")
    df = pd.read_csv(raw_data_path + filename)
    df = df.rename(columns={df.columns[0]: "Date"})
    df["Date"] = pd.to_datetime(df["Date"].astype("str").str.split(" ", n=1).apply(lambda x: x[0]))
    df = df.set_index("Date") 
    df = df.loc[sim_period[0] : sim_period[1]] # Slice only data within simulation period
    
    for data_name in data_names:
        stock_name = filename.split("_")[1]
        stock_data = df[data_name].rename(stock_name).to_frame()
        if data_name not in data:
            data[data_name] = stock_data
        else:
            data[data_name] = data[data_name].merge(stock_data, how='outer', left_index=True, right_index=True)

Processing 100 / 1048 files...
Processing 200 / 1048 files...
Processing 300 / 1048 files...
Processing 400 / 1048 files...
Processing 500 / 1048 files...
Processing 600 / 1048 files...
Processing 700 / 1048 files...
Processing 800 / 1048 files...
Processing 900 / 1048 files...
Processing 1000 / 1048 files...


In [89]:
def save_pickle(pickle_filename: str, data):
    file = open(pickle_filename, 'wb')
    pickle.dump(data, file)
    file.close()

def load_pickle(pickle_filename: str):
    file = open(pickle_filename, 'rb')
    data = pickle.load(file)
    file.close()
    return data

In [92]:
# Cache pre-aggregated stock data into data directory
for data_name in data_names:
    save_pickle(output_data_path + data_name.lower() + ".pickle", data[data_name])

In [95]:
# Compute returns based on close price and cache
returns = data["Close"].pct_change()
save_pickle(output_data_path + "returns.pickle", returns)

In [97]:
# Save data dictionary for convenience
data["returns"] = returns
save_pickle(output_data_path + "data.pickle", data)